# to plate CSY

looking at functiosn to transform from CMM CSY to plate CSY as they differ between new and old code.

In [1]:
import numpy as np

import cmm_error_map.config.config as cf
import cmm_error_map.data_cmpts as dc
import cmm_error_map.mpl_2014.design_matrix_linear as design_old

In [27]:
# get a set of cmm values that agree between old and new code
model_params = dc.model_parameters_dict.copy()
model_params["Txy"] = 1e-6
signs = {"old": 1, "new": -1}

# XY plane
x0, y0, z0 = 250.0, 50.0, 50.0
x0xy, y0xy, z0xy = x0, y0, z0
transform_mat_xy = np.array(
    [
        [1.0, 0.0, 0.0, x0xy],
        [0.0, 1.0, 0.0, y0xy],
        [0.0, 0.0, 1.0, z0xy],
        [0.0, 0.0, 0.0, 1.0],
    ]
)

xt, yt, zt = 0.0, 0.0, -243.4852

# new code
cmm = dc.pmm_866
prb_xy = dc.Probe(title="P0", name="p0", length=np.array([xt, yt, zt]))

mmt = dc.Measurement(
    title="Plate XY",
    name="mmt_00",
    artefact=cf.artefact_models["KOBA 0620"],
    transform_mat=transform_mat_xy,
    probe=prb_xy,
    cmm_nominal=None,
    cmm_dev=None,
    mmt_nominal=None,
    mmt_dev=None,
)
signed_mp = {key: signs["new"] * value for key, value in model_params.items()}
mmt.recalculate(signed_mp, cmm.cmm_model)

# matplotlib code
params = list(model_params.values())
signed_params = [value * signs["old"] for value in params]
dxy, eXYZ, XYZp, XM, plate_nom = design_old.modelled_mmts_XYZ(
    transform_mat_xy, xt, yt, zt, signed_params, verbose=True
)

print(signed_mp)
print(signed_params)
print(
    f"cmm dev difference\n max {(eXYZ[:-1, :] - mmt.cmm_dev).max()}, min {(eXYZ[:-1, :] - mmt.cmm_dev).min()}"
)

print(f"{eXYZ[1, :]=}")
print(f"{mmt.cmm_dev[1,:]=}")


{'Txx': -0.0, 'Txy': -1e-06, 'Txz': -0.0, 'Tyx': -0.0, 'Tyy': -0.0, 'Tyz': -0.0, 'Tzx': -0.0, 'Tzy': -0.0, 'Tzz': -0.0, 'Rxx': -0.0, 'Rxy': -0.0, 'Rxz': -0.0, 'Ryx': -0.0, 'Ryy': -0.0, 'Ryz': -0.0, 'Rzx': -0.0, 'Rzy': -0.0, 'Rzz': -0.0, 'Wxy': -0.0, 'Wxz': -0.0, 'Wyz': -0.0}
[0.0, 1e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cmm dev difference
 max 5.6199511190568874e-14, min -5.2219513435591836e-14
eXYZ[1, :]=array([-0.00025 , -0.000383, -0.000516, -0.000649, -0.000782, -0.00025 ,
       -0.000383, -0.000516, -0.000649, -0.000782, -0.00025 , -0.000383,
       -0.000516, -0.000649, -0.000782, -0.00025 , -0.000383, -0.000516,
       -0.000649, -0.000782, -0.00025 , -0.000383, -0.000516, -0.000649,
       -0.000782])
mmt.cmm_dev[1,:]=array([-0.00025 , -0.000383, -0.000516, -0.000649, -0.000782, -0.00025 ,
       -0.000383, -0.000516, -0.000649, -0.000782, -0.00025 , -0.000383,
       -0.000516, -0.000649, -0.000782, -0.00025 , -0.0

In [28]:
np.testing.assert_allclose(eXYZ[:-1, :], mmt.cmm_dev, atol=1e-6)

In [26]:
# convert to plate CSY via equivalent to old code
# from design_matrix_linear.modelled_mmts_XYZ
XYZp1 = np.dot(np.linalg.inv(transform_mat_xy), XM + eXYZ)
XYZp1[:3, :] = XYZp1[:3, :] - XYZp1[:3, 0:1]
ang = -1.0 * np.arctan2(XYZp1[1, 4], XYZp1[0, 4])
cang = np.cos(ang)
sang = np.sin(ang)
RZ = np.array(
    [
        [cang, -sang, 0.0, 0.0],
        [sang, cang, 0.0, 0.0],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
    ]
)
XYZp1 = np.dot(RZ, XYZp1)

# check it is the same as above
np.testing.assert_allclose(XYZp, XYZp1, atol=1e-6)

# check ball 1 is at (0, 0, 0)
np.testing.assert_allclose(XYZp1[:-1, 0], np.array([0.0, 0.0, 0.0]), atol=1e-6)

# check ball 5 is at (x, 0, 0)
np.testing.assert_allclose(XYZp1[1:3, 4], np.array([0.0, 0.0]), atol=1e-6)

# check ball 21 is at (x, y, 0)
np.testing.assert_allclose(XYZp1[2, 20], 0.0, atol=1e-6)

In [52]:
# convert to plate CSY via new code
mmt_deform = dc.cmm_to_plate_csy(mmt)
mmt_deform

array([[ 0.00000000e+00,  1.33000000e+02,  2.66000000e+02,
         3.99000000e+02,  5.32000000e+02, -1.33000000e-04,
         1.32999867e+02,  2.65999867e+02,  3.98999867e+02,
         5.31999867e+02, -2.66000000e-04,  1.32999734e+02,
         2.65999734e+02,  3.98999734e+02,  5.31999734e+02,
        -3.99000000e-04,  1.32999601e+02,  2.65999601e+02,
         3.98999601e+02,  5.31999601e+02, -5.32000000e-04,
         1.32999468e+02,  2.65999468e+02,  3.98999468e+02,
         5.31999468e+02],
       [ 0.00000000e+00,  1.77634695e-15,  3.55269390e-15,
        -1.77638651e-15, -3.95655732e-20,  1.33000000e+02,
         1.33000000e+02,  1.33000000e+02,  1.33000000e+02,
         1.33000000e+02,  2.66000000e+02,  2.66000000e+02,
         2.66000000e+02,  2.66000000e+02,  2.66000000e+02,
         3.99000000e+02,  3.99000000e+02,  3.99000000e+02,
         3.99000000e+02,  3.99000000e+02,  5.32000000e+02,
         5.32000000e+02,  5.32000000e+02,  5.32000000e+02,
         5.32000000e+02],
    

In [53]:
mmt_dev2 = mmt_deform - mmt.mmt_nominal

In [54]:
dxy.shape


(25, 2)

In [55]:
mmt_dev2[:-1, :].T.shape

(25, 2)

In [56]:
np.testing.assert_allclose(dxy, mmt_dev2[:-1, :].T, atol=1e-6)

In [23]:
for point in XYZp.T:
    print(point[:-1])

[0. 0. 0.]
[1.33000000e+02 1.77634695e-15 0.00000000e+00]
[2.6600000e+02 3.5526939e-15 0.0000000e+00]
[ 3.99000000e+02 -1.77638651e-15  0.00000000e+00]
[ 5.32000000e+02 -3.95655732e-20  0.00000000e+00]
[-1.33e-04  1.33e+02  0.00e+00]
[132.999867 133.         0.      ]
[265.999867 133.         0.      ]
[398.999867 133.         0.      ]
[531.999867 133.         0.      ]
[-2.66e-04  2.66e+02  0.00e+00]
[132.999734 266.         0.      ]
[265.999734 266.         0.      ]
[398.999734 266.         0.      ]
[531.999734 266.         0.      ]
[-3.99e-04  3.99e+02  0.00e+00]
[132.999601 399.         0.      ]
[265.999601 399.         0.      ]
[398.999601 399.         0.      ]
[531.999601 399.         0.      ]
[-5.32e-04  5.32e+02  0.00e+00]
[132.999468 532.         0.      ]
[265.999468 532.         0.      ]
[398.999468 532.         0.      ]
[531.999468 532.         0.      ]


In [32]:
cmm_deform = mmt.cmm_nominal + mmt.cmm_dev

In [34]:
cmm_deform[0:1, :].shape

(1, 25)

In [37]:
t = np.vstack((cmm_deform, np.ones((1, cmm_deform.shape[1]))))
t.shape

(4, 25)

In [38]:
t = np.vstack((cmm_deform, np.ones_like(cmm_deform[0, :])))
t.shape

(4, 25)